<a href="https://colab.research.google.com/github/Xi123580/Product-Information-Conversion/blob/main/Product_Data_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from time import perf_counter
from google.colab import files
import io


pd.set_option('max_columns', None)

# 1. read file
path = 'https://github.com/Xi123580/Product-Information-Conversion/blob/main/%E7%94%A2%E5%93%81%E8%B3%87%E8%A8%8A_%E5%8E%9F%E5%A7%8B%E8%B3%87%E6%96%99_%E6%AC%84%E4%BD%8D%E8%BD%89%E6%8F%9B%E8%A1%A8_%E8%B3%87%E6%96%99%E8%BD%89%E6%8F%9B%E8%A1%A8_20220421_1.xlsx?raw=true'
df_rawData = pd.read_excel(path, sheet_name = 0)
# print(df_rawData.head().to_string())


             IPROD                                     IDESC                IMNOT1       IDRAW  SAPSBU  ILIST IMFRMC IFCI IMCMTH SAPNOR IPURC
0      618003.1566               Spectinomycin Hydrochloride             觀黴素鹽酸鹽標準品  22189-32-8   781.0  10600      .    A     M2      Y   M2D
1     1.00327.5000  HYDROCHLORIC ACID IN ETHANOL FOR MICROSC          鹽酸標準液配製在乙醇溶液         NaN   649.0   8600    IVD    S    NaN    NaN   M2B
2  EPV0045000.1566                  Vancomycin hydrochloride                鹽酸萬古黴素   1404-93-9   823.0   9800     FD    A    NaN    NaN   M2D
3    P3850000.1566  Pseudoephedrine hydrochloride *Precursor  鹽酸假麻黃素標準品,第四級管制先驅原料藥         NaN   823.0  12400    DEA    A    NaN    NaN   M2D
4    Y0000218.1566       Ondansetron hydrochloride dihydrate                鹽酸昂丹司瓊         NaN   823.0   9600     NC    A    NaN    NaN   M2D


In [ ]:
# 2. Scan and fill in the new table according to the conversion rules

# Page: 2
df_column = pd.read_excel(path, sheet_name = 1)
df_column = df_column.drop(columns = '原始資料欄位名稱', axis = 1)

column_EN = []
column_CH = []
Translation = {}
for i in range(len(df_column.columns[:])):
  column_EN.append(df_column.columns[i])
  column_CH.append(df_column.values[0][i])
  Translation[column_EN[i]] = column_CH[i]
# print(Translation)

{'IPROD': '產品編號', 'IDESC': '英文名稱', 'IMNOT1': '中文名稱', 'IDRAW': 'CAS Number', 'SAPSBU': '產品線', 'ILIST': '未稅定價', 'IMFRMC': '法規需求', 'IFCI': '運送方式', 'IMCMTH': '注意事項', 'SAPNOR': '停產資訊', 'IPURC': '產品分類'}


In [ ]:
# Page: 3~8
def initial_sheet(Page:int):
  df = pd.read_excel(path, sheet_name = Page, usecols = "A, B")
  df.rename(columns={'原始資訊': 'Keys', '網頁呈現字眼': 'Values'}, inplace=True)
  df = df.drop(0)
  df = df.reset_index()
  df = df.drop(columns="index")

  column_SAPSBU = []
  column_Product = []
  ConvertDict = {}
  for i in range(len(df.values[:])):
    column_SAPSBU.append(df.values[i][0])
    column_Product.append(df.values[i][1])
    ConvertDict[column_SAPSBU[i]] = column_Product[i]
  return(ConvertDict)

In [ ]:
Rule_SAPSBU = initial_sheet(2)
Rule_IMFRMC = initial_sheet(3)
Rule_IMCMTH = initial_sheet(4)
Rule_IFCI = initial_sheet(5)
Rule_SAPNOR = initial_sheet(6)
Rule_IPURC = initial_sheet(7)
# print(Rule_IFCI)
# print(Rule_IMCMTH)

{'A': '空運', 'AIR': '空運', 'C': '空運', 'L': '請洽默克窗口', 'N': '請洽默克窗口', nan: '請洽默克窗口', 'P': '空運', 'PAX': '空運', 'S': '海運', 'SEA': '海運', 'Y': '請洽默克窗口'}
{7: '已停產', 99: '需法規審核', 'M1': '已停產', 'M2': '已停產', 'YY': '已停產'}


In [ ]:
# upload file ( .xlsx )
print("請上傳Excel檔 (副檔名須為.xlsx): ")
uploaded = files.upload()
# Start_1 = perf_counter()
filename = next(iter(uploaded))
uploaded = pd.read_excel(io.BytesIO(uploaded.get(f'{filename}')), dtype=str)
print(uploaded.head(3).to_string())

請上傳Excel檔 (副檔名須為.xlsx): 


Saving IIM_Product_List (1).xlsx to IIM_Product_List (1).xlsx
         IPROD                                     IDESC                                    IMNOT1      IDRAW SAPSBU ILIST IMFRMC IFCI IMCMTH SAPNOR IPURC
0  185450-100G  Phenol, unstabilized, ReagentPlus, >=99%  GUS: PHENOL, UNSTABILIZED, REAGENTPLUS(R   108-95-2    774  2900    NaN    A    NaN    NaN   M2A
1    I2215-25G                            5-INDANOL, 99%                       GUS: 5-INDANOL, 99%  1470-94-6    774  3200      .    A    NaN    NaN   M2A
2  110388-25ML                    BUTYLCYCLOHEXANE, 99+%                    BUTYLCYCLOHEXANE, 99+%  1678-93-9    774  2600      .    A    NaN    NaN   M2A


In [ ]:
# 3. build table
# Start_2 = perf_counter()
New_df = uploaded.copy()

print(New_df.keys())
print(New_df.shape)
Number_columns = New_df.shape[1]
Number_row = New_df.shape[0]
New_df = New_df.fillna(0)

# figure out table
for col in range(Number_columns):
  Col_Name = df_rawData.keys()[col]
  if Col_Name == "SAPSBU":
    # New_df[Col_Name] = New_df[Col_Name].apply(int)
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_SAPSBU:
        x = Rule_SAPSBU[data]
        New_df.at[row, Col_Name] = x
      else:
        New_df.at[row, Col_Name] = "-"
  elif Col_Name == "IMFRMC":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IMFRMC:
        x = Rule_IMFRMC[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "無管制_但請再確認注意事項"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "IFCI":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IFCI:
        x = Rule_IFCI[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "-"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "IMCMTH":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IMCMTH:
        x = Rule_IMCMTH[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "-"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "SAPNOR":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_SAPNOR:
        x = Rule_SAPNOR[data]
        New_df.at[row, Col_Name] = x
      else:
        New_df.at[row, Col_Name] = "-"
  elif Col_Name == "IPURC":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name]
      if data in Rule_IPURC:
        x = Rule_IPURC[data]
        New_df.at[row, Col_Name] = x
      elif data == "0":
        New_df.at[row, Col_Name] = "Blue"
      else:
        New_df.at[row, Col_Name] = "Blue"    
  else :
    pass

# Change Title
New_df = New_df.rename(Translation, axis=1)

print(New_df.head().to_string())

Index(['IPROD', 'IDESC', 'IMNOT1', 'IDRAW', 'SAPSBU', 'ILIST', 'IMFRMC',
       'IFCI', 'IMCMTH', 'SAPNOR', 'IPURC'],
      dtype='object')
(277900, 11)
          產品編號                                      英文名稱                                      中文名稱  CAS Number                              產品線  未稅定價           法規需求 運送方式 注意事項 停產資訊 產品分類
0  185450-100G  Phenol, unstabilized, ReagentPlus, >=99%  GUS: PHENOL, UNSTABILIZED, REAGENTPLUS(R    108-95-2  LSC_774 Chemical Synthesis - RC  2900  無管制_但請再確認注意事項   空運    -    -  Red
1    I2215-25G                            5-INDANOL, 99%                       GUS: 5-INDANOL, 99%   1470-94-6  LSC_774 Chemical Synthesis - RC  3200  無管制_但請再確認注意事項   空運    -    -  Red
2  110388-25ML                    BUTYLCYCLOHEXANE, 99+%                    BUTYLCYCLOHEXANE, 99+%   1678-93-9  LSC_774 Chemical Synthesis - RC  2600  無管制_但請再確認注意事項   空運    -    -  Red
3    661384-1G  2-Iodoxybenzoic acid, stabilized (45 wt.  2-IODOXYBENZOIC ACID, STABILIZED (45 WT&  61717-8

In [ ]:
# Convert to large table and output
Path_beta = "https://raw.githubusercontent.com/Xi123580/Product-Information-Conversion/main/beta.csv"
Beta = pd.read_csv(Path_beta, header=0)
Beta = pd.DataFrame(columns=Beta.columns)
# print(Beta)

Beta["屬性 1 值"] = Beta["貨號"] = New_df["產品編號"]
Beta["屬性 2 值"] = Beta["名稱"] = Beta["簡短內容說明"] = New_df["英文名稱"]
Beta["屬性 3 值"] = Beta["描述"] = New_df["中文名稱"]
Beta["屬性 4 值"] = New_df["CAS Number"]
Beta["屬性 5 值"] = New_df["產品線"]
Beta["屬性 6 值"] = New_df["法規需求"]
Beta["屬性 7 值"] = New_df["注意事項"]
Beta["屬性 8 值"] = New_df["運送方式"]
Beta["屬性 9 值"] = New_df["停產資訊"]
Beta["屬性 10 值"] = Beta["分類"] = New_df["產品分類"]
Beta["原價"] = New_df["未稅定價"]
Beta["已發佈"] = Beta["有庫存？"] = Beta["允許客戶評論嗎？"] = "1"
Beta["是特色商品？"] = Beta["允許無庫存下單嗎？"] = Beta["單獨銷售?"] = Beta["位置"] = "0"
Beta["目錄的可見度"] = "visible"
Beta["稅金狀態"] = "taxable"

for X in range(1,11,1):
  Beta[f"屬性 {X} 可見"] = "1"
  Beta[f"屬性 {X} 全域"] = "1"

PropertyNname = ["產品編號", "英文名稱", "中文名稱", "CAS Number", "產品線",
          "法規需求", "注意事項", "運送方式", "停產資訊", "產品分類"]
for X in range(1,11,1):
  Beta[f"屬性 {X} 名稱"] = PropertyNname[(X-1)]


# print(New_df.keys())
# print(Beta.head().to_string())
print(Beta.shape)

# End = perf_counter()
# print(End-Start_1)
# print(End-Start_2)

(277900, 80)


In [ ]:
# 4. save file as .csv and output.
for n in range(4):#int(Number_row/10000)):
  Beta_5000 = Beta.iloc[n*5000:(n+1)*5000]
  Beta_5000.to_csv(path_or_buf = f"產品資訊_Zh-TW_{n+1}.csv", sep = ",", na_rep = "",
          index = False, encoding = "utf-8-sig")
  files.download(f"產品資訊_Zh-TW_{n+1}.csv")
print("Finish")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Finish
